# CLVO Netrwork development

## Initialization

In [ ]:
# PyTorch imports
import torch

# Other external package imports
import matplotlib.pyplot as plt
import numpy as np

# Project module imports
from odometry.vo_datasets import FlowKittiDataset
from odometry.clvo import CLVO
from utils.helpers import log, transform
from utils.arguments import Arguments
from utils.normalization import FlowStandardization

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100

# Instantiating arguments object for optical flow module
args = Arguments.get_arguments()

# Instantiating dataset and dataloader
batch_size = 16
sequence_length = 1
dataset = FlowKittiDataset(args.data_path, ["00"], augment=False, sequence_length=1)

model = CLVO().to(args.device)
model.load_state_dict(torch.load("checkpoints/clvo_generalization4_1.pth", map_location=args.device))
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
log("Trainable parameters:", trainable_params)

normalization =  FlowStandardization().eval()

## Running inference on 00 and collecting output

In [ ]:
with torch.no_grad():
    model.to(args.device)
    model.eval()

    translations = []
    rotations = []
    #resize = Resize((376, 1248))
    use_model = True

    count = 0
    for i in range(0, len(dataset), 1):
        flow, rot, tr = dataset[i]
        if use_model:
            flow = normalization(flow.unsqueeze(0).to(args.device))

            rot, tr = model(flow)
        rot = rot.detach()
        tr = tr.detach()
        
        rotations.append(rot)        
        translations.append(tr)
        
        #for j in range(len(rot)):
        #    translation = tr[j].squeeze()
        #    rotation = rot[j].squeeze()

            #translations.append(translation)
            #rotations.append(rotation)

        print('    ', end='\r')
        print(count, end='')
        count = count+1

    print()
    log("Data loading Done!")
    log("Rot length: ", len(rotations))
    log("Tr length: ", len(translations))

## Rel->Abs coordinate conversion and plotting results

In [ ]:
homogenous = []

instance_num = len(rotations)
for i in range(instance_num):
    homogenous.append(transform(rotations[i].squeeze(), translations[i].squeeze()))

global_scale = []
global_scale.append(homogenous[0])
for i in range(1, instance_num):
    global_scale.append(torch.matmul(global_scale[i-1], homogenous[i]))
    
global_scale = torch.stack(global_scale, dim=0)
global_pos = global_scale[:, :3, -1]


# Plotting results
numpy_poses = []
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100

for i in range(len(global_scale)):
    numpy_poses.append((np.array(global_scale[i][:3, :].view(12).numpy())))
numpy_poses = np.stack(numpy_poses, axis=0)

np.savetxt("results.txt", numpy_poses)
reloaded_poses = np.loadtxt("results.txt")

X = np.array([p[3] for p in reloaded_poses])
Z = np.array([p[-1] for p in reloaded_poses])

plt.plot(X, Z)
plt.show()

## Comparison with Ground Truth

In [ ]:
DATA_PATH = args.data_path + "/dataset/poses/00.txt"
true_pos = np.loadtxt(DATA_PATH)

X_gt, Y_gt, Z_gt = true_pos[:, 3], true_pos[:, 7], true_pos[:, 11]
X, Y, Z = global_pos[:, 0], global_pos[:, 1], global_pos[:, 2]

plt.plot(X, Z, label="pred")
plt.plot(X_gt, Z_gt, label="gt")
plt.legend()
plt.show()

plt.plot(X, label="pred")
plt.plot(X_gt, label="gt")
plt.legend()
plt.show()

plt.plot(Y, label="pred")
plt.plot(Y_gt, label="gt")
plt.legend()
plt.show()

plt.plot(Z, label="pred")
plt.plot(Z_gt, label="gt")
plt.legend()
plt.show()

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100

X, Y, Z = global_pos[:, 0], global_pos[:, 1], global_pos[:, 2]

plt.plot(X, Z)
plt.show()
plt.plot(X)
plt.show()
plt.plot(Y)
plt.show()
plt.plot(Z)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from odometry.vo_datasets import KittiOdometryDataset
from utils.arguments import Arguments
from GMA.core.utils import flow_viz
import torch
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

dataset = KittiOdometryDataset(args.data_path, "00", precomputed_flow=True, sequence_length=1)

def viz(flo):
    flo = flo[0].permute(1, 2, 0).cpu().numpy()
    # map flow to rgb image
    flo = flow_viz.flow_to_image(flo)
    return flo
    

for i in range(4):
    im1, im2, flow, rot, tr = dataset[i]
    flo_im = viz(flow[0].squeeze)
    plt.imshow(flo_im)
    plt.show()
    #DATA_PATH = None # TODO Define save path
    #plt.imsave(DATA_PATH + str(i) + ".png", flo_im)
    #print(i)